In [1]:
# Autoreload possibly interferes with IntelliJ debugging
%reload_ext autoreload
%autoreload 2
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
log = lambda msg: logging.info(msg)


In [2]:
import pandas as pd
import os

input_dir = "C:\\dev\\play\\brainwave-data"
stats_df = pd.read_csv(input_dir + os.path.sep + "stats.csv")

In [ ]:
import json
from datetime import datetime
import os
import run_feature_pipeline
errors = []

dfs = []
sleep_json_dfs = []
# Could get these working later
skip_list = ['2024-07-23-22-40-25', '2024-07-28-22-29-49', '2024-09-18-21-25-08', '2024-09-18-21-28-11', '2024-09-19-21-29-42']


def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

for root, dirs, files in os.walk(input_dir):
    for idx, dir_name in enumerate(dirs):
        input_file = os.path.join(root, dir_name, "raw.fif")
        if dir_name in skip_list:
            log(f"Skipping {idx} of {len(dirs)}: " + input_file)
            continue


        json_file = os.path.join(root, dir_name, "raw.sleep.json")
        try:
            if os.path.exists(json_file):
                with open(json_file, 'r') as f:
                    data = json.load(f)
                    flat_data = flatten_json(data)
                    flat_data['dir_name'] = dir_name
                    sleep_json_dfs.append(pd.DataFrame([flat_data]))
        except Exception as e:
            log("Error processing file: " + json_file)
            errors.append("Error processing file: " + json_file + " - " + str(e))
            log(e)

        try:
            log("Processing file: " + input_file)

            if os.path.exists(input_file):
                # day_and_night_of_date = datetime.strptime(dir_name, "%Y-%m-%d")
                day_and_night_of_date = datetime.strptime(dir_name, "%Y-%m-%d-%H-%M-%S")
                day_and_night_of = day_and_night_of_date.date().isoformat()
                %%capture captured
                yasa_df = run_feature_pipeline.cached_pipeline(log, input_file, stats_df)
                yasa_df['dayAndNightOf'] = day_and_night_of
                dfs.append(yasa_df)
        except Exception as e:
            log("Error processing file: " + input_file)
            errors.append("Error processing file: " + input_file + " - " + str(e))
            log(e)

for err in errors:
    log(err)

sleep_json_df = pd.concat(sleep_json_dfs)

yasa_df = pd.concat(dfs)
yasa_df

2024-11-12 17:11:12,648 - INFO - Writing to C:\dev\play\brainwave-data\2024-10-04-21-13-39\raw.sleep_stages.csv
2024-11-12 17:11:12,864 - INFO - Memory Usage: 6656.37 MB GC to 6656.37 MB
2024-11-12 17:11:12,865 - INFO - Processing sleep statistics
2024-11-12 17:11:13,072 - INFO - Memory Usage: 6656.37 MB GC to 6656.37 MB
2024-11-12 17:11:13,073 - INFO - Processing sleep stability
2024-11-12 17:11:13,308 - INFO - Memory Usage: 6656.37 MB GC to 6656.37 MB


Setting up band-pass filter from 0.3 - 1.5 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.20 Hz (-6 dB cutoff frequency: 0.20 Hz)
- Upper passband edge: 1.50 Hz
- Upper transition bandwidth: 0.20 Hz (-6 dB cutoff frequency: 1.60 Hz)
- Filter length: 4125 samples (16.500 s)



2024-11-12 17:11:19,667 - INFO - Memory Usage: 6699.30 MB GC to 6666.54 MB
2024-11-12 17:11:19,670 - INFO - YASA all done!
2024-11-12 17:11:19,909 - INFO - Memory Usage: 6602.50 MB GC to 6602.50 MB
C:\dev\play\brainwave-processor\scaling.py:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[renamed] = df[col]
C:\dev\play\brainwave-processor\scaling.py:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[renamed] = df[col]
C:\dev\play\brainwave-processor\scaling.py:77: PerformanceWarning: DataFrame is highly fragmented.  This i

1/1 [==============================] - 1s 655ms/step
Saved microwakings to: C:\dev\play\brainwave-data\2024-10-04-21-13-39\raw.microwakings.csv
Channel
(1, 8131377)
0
Padding needed 2
8131377
Upsampled channel: 8131377
Orig (1, 8131377)
(1, 8131377)
1
8131377


2024-11-12 17:11:26,524 - INFO - Exporting to: C:\dev\play\brainwave-data\2024-10-04-21-13-39\raw.edf
2024-11-12 17:11:27,318 - INFO - All done! C:\dev\play\brainwave-data\2024-10-04-21-13-39\raw.fif
2024-11-12 17:11:27,331 - INFO - Processing file: C:\dev\play\brainwave-data\2024-10-05-21-09-46\raw.fif
2024-11-12 17:11:27,333 - INFO - Loading cached file C:\dev\play\brainwave-data\2024-10-05-21-09-46\raw.with_features.csv
2024-11-12 17:11:27,382 - INFO - Cached file C:\dev\play\brainwave-data\2024-10-05-21-09-46\raw.with_features.csv mod date 2024-11-08 18:39:48.763317 is < 2024-11-11 13:30:00, rebuilding
2024-11-12 17:11:27,383 - INFO - Loading MNE file C:\dev\play\brainwave-data\2024-10-05-21-09-46\raw.fif
2024-11-12 17:11:27,383 - INFO - Reading file C:\dev\play\brainwave-data\2024-10-05-21-09-46\raw.fif
2024-11-12 17:11:27,995 - INFO - Finished reading file C:\dev\play\brainwave-data\2024-10-05-21-09-46\raw.fif
2024-11-12 17:11:28,730 - INFO - Start date: 2024-10-05 20:09:46.50000

Setting up band-pass filter from 0.3 - 1.5 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.20 Hz (-6 dB cutoff frequency: 0.20 Hz)
- Upper passband edge: 1.50 Hz
- Upper transition bandwidth: 0.20 Hz (-6 dB cutoff frequency: 1.60 Hz)
- Filter length: 4125 samples (16.500 s)



2024-11-12 17:11:35,398 - INFO - Memory Usage: 6691.82 MB GC to 6691.82 MB
2024-11-12 17:11:35,398 - INFO - Running YASA
2024-11-12 17:11:35,592 - INFO - Memory Usage: 6691.82 MB GC to 6691.82 MB
2024-11-12 17:11:39,449 - WARNING - Hypnogram is SHORTER than data by 25.35 seconds. Padding hypnogram with last value to match data.size.
2024-11-12 17:11:48,149 - INFO - Writing to C:\dev\play\brainwave-data\2024-10-05-21-09-46\raw.sleep_stages.csv
2024-11-12 17:11:48,357 - INFO - Memory Usage: 6869.95 MB GC to 6869.95 MB
2024-11-12 17:11:48,357 - INFO - Processing sleep statistics
2024-11-12 17:11:48,553 - INFO - Memory Usage: 6869.95 MB GC to 6869.95 MB
2024-11-12 17:11:48,554 - INFO - Processing sleep stability
2024-11-12 17:11:48,752 - INFO - Memory Usage: 6869.95 MB GC to 6869.95 MB


Setting up band-pass filter from 0.3 - 1.5 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.20 Hz (-6 dB cutoff frequency: 0.20 Hz)
- Upper passband edge: 1.50 Hz
- Upper transition bandwidth: 0.20 Hz (-6 dB cutoff frequency: 1.60 Hz)
- Filter length: 4125 samples (16.500 s)



2024-11-12 17:11:56,112 - INFO - Memory Usage: 6913.62 MB GC to 6878.54 MB
2024-11-12 17:11:56,114 - INFO - YASA all done!
2024-11-12 17:11:56,344 - INFO - Memory Usage: 6813.89 MB GC to 6813.89 MB
C:\dev\play\brainwave-processor\scaling.py:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[renamed] = df[col]
C:\dev\play\brainwave-processor\scaling.py:77: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[renamed] = df[col]
C:\dev\play\brainwave-processor\scaling.py:77: PerformanceWarning: DataFrame is highly fragmented.  This i

1/1 [==============================] - 1s 758ms/step


In [ ]:
sleep_json_df

In [ ]:
[col for col in sleep_json_df.columns if 'sigmaabsaa_s' in col]

In [ ]:
sleep_json_df[['Fpz_eeg_sigmaabs_aa_s', 'dir_name']]

In [ ]:
sleep_json_df['SlowWaves_Fpz-M1_PTP'].describe()
